In [2]:
from sympy import sin, cos, Matrix
from sympy.abc import x, y, z
from sympy import diff

import numpy as np

## Symbolic

In [3]:
X = Matrix([x, y, z])
F = Matrix([x*y + 2*z*x, 5*x + sin(y) + 3*z])
f = F[0, 0]

In [4]:
def gradient(f, X):
    """Returns a tuple of length f.ndim where each entry represents a the numerical derivative Gj = df/dxj"""
    return np.asarray([diff(f, x) for x in X])

In [5]:
def jacobian(F, X):
    """Returns array of shape len(f) x len(x) where each entry represents the first-order derivative Jij = dfi/dxj"""
    return np.asarray([[diff(f, x) for x in X] for f in F])

In [6]:
def hessian(f, X):
    """Returns array of shape len(f) x len(x) where each entry represents the first-order derivative Jij = dfi/dxj"""
    return jacobian(gradient(f, X), X).T

In [7]:
G = gradient(f, X)
print(G.shape)
print(G)

(3,)
[y + 2*z x 2*x]


In [8]:
J = jacobian(F, X)
print(J.shape)
print(J)

(2, 3)
[[y + 2*z x 2*x]
 [5 cos(y) 3]]


In [9]:
H = hessian(f, X)
print(H.shape)
print(H/np.max(H))

(3, 3)
[[0 1/2 1]
 [1/2 0 0]
 [1 0 0]]


## Numeric

In [10]:
x = np.linspace(-0.001, 0.00101, 3).reshape(-1, 1, 1)
y = np.linspace(-0.001, 0.00101, 3).reshape(1, -1, 1)
z = np.linspace(-0.001, 0.00101, 3).reshape(1, 1, -1)

F = np.array([x*y + 2*z*x, 5*x + np.sin(y) + 3*z]) # Need be size 3*X and shape f_i, x_1, x_2,..., x_j
X1 = np.asarray([x, y, z])
X2 = np.asarray([np.squeeze(x) for x in X1])
f = F[0]

In [11]:
def gradient(f, X):
    """Returns a tuple of length f.ndim where each entry represents a the numerical derivative Gj = df/dxj"""
    return np.asarray([(np.diff(f, 1, j)/np.diff(x, 1, j)) for j, x in enumerate(X)])

def gradient2(f, *X):
    return np.asarray(np.gradient(f, *X))

In [12]:
def jacobian(F, X):
    """Returns array of shape len(f) x len(x) where each entry represents the first-order derivative Jij = dfi/dxj"""
    return np.asarray([[np.diff(f, 1, j)/np.diff(x, 1, j) for j, x in enumerate(X)] for f in F])

def jacobian2(F, *X):
    return np.stack([np.gradient(f, *X) for f in F]) 

In [13]:
def hessian(f, *X):
    """Returns array of shape len(f) x len(x) where each entry represents the first-order derivative Jij = dfi/dxj"""
    return np.swapaxes(jacobian2(gradient2(f, *X), *X), 0, 1)

In [16]:
G = gradient(f, X1)
print(G.shape)
print(G[0].shape, G[1].shape, G[2].shape)
G[0][0, 0, 0], G[1][0, 0, 0], G[2][0, 0, 0]

(3,)
(2, 3, 3) (3, 2, 3) (3, 3, 2)


(-0.0029999999999999996, -0.001, -0.002)

In [18]:
G = gradient2(f, *X2)
print(G.shape)
print(G[0].shape, G[1].shape, G[2].shape)
G[:, 0, 0, 0]

(3, 3, 3, 3)
(3, 3, 3) (3, 3, 3) (3, 3, 3)


array([-0.003, -0.001, -0.002])

In [19]:
J = jacobian(F, X1)
print(J.shape)
print(J[0, 0].shape, J[0, 1].shape, J[0, 2].shape)
print(J[1, 0].shape, J[1, 1].shape, J[1, 2].shape)

(2, 3)
(2, 3, 3) (3, 2, 3) (3, 3, 2)
(2, 3, 3) (3, 2, 3) (3, 3, 2)


In [23]:
J = jacobian2(F, *X2)
print(J.shape)
J[0, :, 0, 0, 0]

(2, 3, 3, 3, 3)


array([-0.003, -0.001, -0.002])

In [24]:
H = hessian(f, *X2)
print(H.shape)
H[:, :, 2, 2, 2]/np.max(H[:, :, 2, 2, 2])

(3, 3, 3, 3, 3)


array([[  0.00000000e+00,   5.00000000e-01,   1.00000000e+00],
       [  5.00000000e-01,   0.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,  -1.07880813e-16,   0.00000000e+00]])

## Numeric with Sub-Sweep

In [25]:
x = np.linspace(-0.001, 0.00101, 3).reshape(-1, 1, 1, 1, 1)
y = np.linspace(-0.001, 0.00101, 3).reshape(1, -1, 1, 1, 1)
z = np.linspace(-0.001, 0.00101, 3).reshape(1, 1, -1, 1, 1)
s1 = np.linspace(-0.001, 2, 9).reshape(1, 1, 1, -1, 1)
s2 = np.linspace(-0.001, 2, 11).reshape(1, 1, 1, 1, -1)
num_vars = 3
axes = np.arange(0, num_vars)

F = np.array([x*y + 2*z*x + s1 + s2, 5*x + np.sin(y) + 3*z + s1 + s2]) # Need be size 3*X and shape f_i, x_1, x_2,..., x_j
f = F[0]
X1 = np.asarray([x, y, z, s1, s2])
X2 = np.asarray([np.squeeze(x) for x in X1])
print(f.shape, X1.shape, X2.shape)

(3, 3, 3, 9, 11) (5,) (5,)


In [26]:
def gradient2(f, *X, axis=None):
    return np.asarray(np.gradient(f, *X, axis=axis))

In [27]:
def jacobian2(F, *X, axis=None):
    return np.stack([np.gradient(f, *X, axis=axis) for f in F]) 

In [28]:
def hessian(f, *X, axis=None):
    """Returns array of shape len(f) x len(x) where each entry represents the first-order derivative Jij = dfi/dxj"""
    return np.swapaxes(jacobian2(gradient2(f, *X, axis=axis), *X, axis=axis), 0, 1)

In [29]:
G = gradient2(f, *X2[0:num_vars], axis=tuple(axes))
print(G.shape)
print(G[0].shape, G[1].shape, G[2].shape)
G[:, 0, 0, 0, 0, 0]

(3, 3, 3, 3, 9, 11)
(3, 3, 3, 9, 11) (3, 3, 3, 9, 11) (3, 3, 3, 9, 11)


array([-0.003, -0.001, -0.002])

In [32]:
J = jacobian2(F, *X2[0:num_vars], axis=tuple(axes))
print(J.shape)
J[0, :, 0, 0, 0, 0, 0]

(2, 3, 3, 3, 3, 9, 11)


array([-0.003, -0.001, -0.002])

In [33]:
H = hessian(f, *X2[0:num_vars], axis=tuple(axes))
print(H.shape)
H[:, :, 2, 2, 2, 0, 0]/np.max(H[:, :, 2, 2, 2, 0, 0])

(3, 3, 3, 3, 3, 9, 11)


array([[ -2.14682818e-13,   5.00000000e-01,   1.00000000e+00],
       [  5.00000000e-01,   0.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00,  -1.07449290e-13]])

## Where/When to Calculate Gradient, Jacobian, Hessian

### Multi-Objective and Multi-Variate
1. The Hessian is scalar function (single objective/multi-variate)
2. The Jaacobian is a vector function (multi-objective/multi-variate)
3. The Numerical Jacobian uses a scalar Gradient => (single objective/multi-variate)

Therefore it is best to only do single objective minimizations.

### When Each be calcualted
1. The Hessian requires 3*num_variables measurements (2nd Order).
2. The Jacobian requires 2*num_variables measurements (1st Order).
3. The Numerical Jacobian uses a scalare Gradient that uses cnetral difference equations => 3*num_variables

So, if the minimization uses a Jacobian/Hessian it must record atleast 3 measurements.

### How to get the first 3 measurements
1. Repeat the first measurement and rely on measurement noise to vary parameters (Infinitesimally small derivatives).
2. Perform and initial random sweep of 3 points for each variable (bounds??)

Since a measurement always has noise, the first option is easier and provides a good initial Jacobian/Hessian. It could be implementd by simply refusing to evaluate the next guess until 3 measurements are completed.

### Where should the Function/Jacobian/Hessian
1. We know this will likely always be limited to a single objective function.
2. Requires access to X (indep_map), F (objective function evaluation of dataset vars), over all realtime sweeps.
3. The evaluation could take place:
    a) In SweepIter.__next__
      - self.indep_map, NO DATASET, minimize.goal, self.step = realtime sweep size
    b) In Minimize.__next__
      - Abstract.indep_map, datagroup_model.dataset, self.goal, Abstract.indep_map["realtime"]
    c) In Objective.__next__, Objective.__jac__, Objective.__hess__
      - generator function for each realtime index. 
      - Could access Mnimize.__call__, __jac__, __hess__, which is a class funcs Goal.__call__, __jac__, __hess__
      - Need to index in each instance goal

### Actions
1. Calculate Goal.Call/Jacobian/Hessian during AbstractMinimize.__next__ (Done)
2. Remove index into Goal.call/jacobian/hessian (Done)
3. Store realtime_index in Goal (Done)
4. Replace Goal input parameters with Minimize.__call__, Minimize__jac__, Minimize.__hess__(Done)
5. Don't stop optimizing if some goals reach completion.  (Not Done)
6. Find some way to bypass __jac__/__hess__ if more measurements are needed (Do Something with goal.order)
